# Imports

Tutorial https://riverml.xyz/0.14.0/examples/bike-sharing-forecasting/ 

SEA https://riverml.xyz/0.14.0/api/datasets/synth/SEA/

Classifiers are there

In [16]:
import pandas as pd
from river.datasets import synth

from river import compose
from river import metrics
from river import utils
from river import linear_model, naive_bayes, tree
from river import preprocessing

# Create flow

In [8]:
class DriftingFlow:

    def __init__(self, noise, seed, variant_1=0, variant_2=1, switch_after=10000, iters=40000):
        self.generator_1 = synth.SEA(noise=noise, variant=variant_1, seed=seed)
        self.generator_2 = synth.SEA(noise=noise, variant=variant_2, seed=seed)
        self.switch_after = switch_after
        self.counter = 0
        self.data = list(self.generator_1.take(switch_after)) + list(self.generator_2.take(iters-switch_after))
    
    def take(self):
        '''Returns (x, y) pair of dataset. x - dict, y - bool'''
        self.counter += 1
        return self.data[self.counter]

In [9]:
def evaluate_model(model, data_generator, iterations=30000, print_every=1000):
    y_trues, y_predicts = [], []
    metr = [
        utils.Rolling(metrics.Accuracy(), 1000)
        , utils.Rolling(metrics.ConfusionMatrix(), 1000)
    ]
    
    
    for i in range(iterations):
        x, y = data_generator.take()
        y_pred = model.predict_one(x)
        model.learn_one(x, y)
        #display(x, y)
        
        metr[0].update(y, y_pred)
        metr[1].update(y, y_pred)
        
        if (i-1) % print_every == 0:
            print('='*20)
            print(f'Iteration {i}')
            print(f'Accuracy: {metr[0].get()}')
            print(f'Confusion matrix\n')
            display(metr[1])
        
        y_trues.append(y)
        y_predicts.append(y_pred)
        
    return y_trues, y_predicts

# Evaluate

## NaiveBayes

In [13]:
data = DriftingFlow(1.5e4, 200)

In [14]:
model = compose.Pipeline(
    #('scale', preprocessing.StandardScaler()),
    ('lin_reg', naive_bayes.MultinomialNB())
)

In [15]:
y_tr, y_pr = evaluate_model(model, data)

Iteration 1
Accuracy: 0.0
Confusion matrix



TypeError: '<' not supported between instances of 'NoneType' and 'bool'

Iteration 1001
Accuracy: 0.746
Confusion matrix



        False   True  
False     581     95  
 True     159    165  

Iteration 2001
Accuracy: 0.767
Confusion matrix



...

        False   True  
False     519     86  
 True     223    172  

Iteration 28001
Accuracy: 0.7
Confusion matrix



        False   True  
False     484    103  
 True     197    216  

Iteration 29001
Accuracy: 0.715
Confusion matrix



        False   True  
False     511    101  
 True     184    204  

## Hoeffding tree

In [18]:
data = DriftingFlow(1.5e4, 200)

In [17]:
model = compose.Pipeline(
    #('scale', preprocessing.StandardScaler()),
    ('tree', tree.HoeffdingTreeClassifier())
)

In [19]:
y_tr, y_pr = evaluate_model(model, data)

Iteration 1
Accuracy: 0.0
Confusion matrix



TypeError: '<' not supported between instances of 'NoneType' and 'bool'

Iteration 1001
Accuracy: 0.927
Confusion matrix



        False   True  
False     672      4  
 True      69    255  

Iteration 2001
Accuracy: 0.945
Confusion matrix



...

        False   True  
False     592     13  
 True       7    388  

Iteration 28001
Accuracy: 0.988
Confusion matrix



        False   True  
False     581      6  
 True       6    407  

Iteration 29001
Accuracy: 0.979
Confusion matrix



        False   True  
False     601     11  
 True      10    378  

## Perceptron

In [20]:
data = DriftingFlow(1.5e4, 200)

In [21]:
model = compose.Pipeline(
    #('scale', preprocessing.StandardScaler()),
    ('tree', linear_model.Perceptron())
)

In [22]:
y_tr, y_pr = evaluate_model(model, data)

Iteration 1
Accuracy: 0.5
Confusion matrix



        False   True  
False       1      0  
 True       1      0  

Iteration 1001
Accuracy: 0.763
Confusion matrix



        False   True  
False     587     89  
 True     148    176  

Iteration 2001
Accuracy: 0.841
Confusion matrix



...

        False   True  
False     569     36  
 True      43    352  

Iteration 28001
Accuracy: 0.947
Confusion matrix



        False   True  
False     563     24  
 True      29    384  

Iteration 29001
Accuracy: 0.916
Confusion matrix



        False   True  
False     572     40  
 True      44    344  

## ELM

In [23]:
data = DriftingFlow(1.5e4, 200)